In [56]:
video_path = 'GOPR1077_tr.mp4'
model_path = 'best.pt' # path to .pt file from YOLOv10 notebook

# dimensions of photos the model was trained on
IMAGE_WIDTH = 416
IMAGE_HEIGHT = 416

In [57]:
!pip install -q av pillow

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [58]:
from ultralytics import YOLOv10
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av, os
from PIL import Image
import time

model = YOLOv10(model_path)
bba = sv.BoundingBoxAnnotator()
la = sv.LabelAnnotator()

In [59]:
def run_pred(input_img, label_annotator = la, bounding_box_annotator = bba, verbose = False):
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model(cont_img, conf=0.10, verbose = verbose)[0]
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)
    return(annotated_image)
# im_pil = Image.open("oyster_test.jpg")
# image = np.array(im_pil)https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov10-object-detection-on-custom-dataset.ipynb
# image_2 = np.array(Image.open('problem_photo_small.jpg'))
# print(im_pil.size)
# ann_img = run_pred(image_2)
# sv.plot_image(ann_img)

In [103]:
container = av.open(video_path)
stream_vid = container.streams.video[0]

fname = video_path.rsplit('/', 1)[-1]

per_index = fname.index('.')

out_path = f'{fname[:per_index]}_annotated.mp4'


start_time = time.time()

outp = av.open(out_path, 'w')
codec_name = stream_vid.codec_context.name
fps = stream_vid.codec_context.rate
output_stream = outp.add_stream(codec_name, str(fps))
output_stream.width = IMAGE_WIDTH
output_stream.height = IMAGE_HEIGHT
output_stream.pix_fmt = stream_vid.codec_context.pix_fmt

for index, frame in enumerate(container.decode(stream_vid)):
    pil_img = frame.to_image()
    np_img = np.array(pil_img)
    np_img_resize = cv2.resize(np_img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    np_rot = np_img_resize[:, :, ::-1]
    small_pil_img = Image.fromarray(np_rot)
    np_image_2 = np.array(small_pil_img)
    an_mg = run_pred(np_image_2, verbose = False)
# _______________________________________
#     if index == 17: # debug a single frame
#         print('________________________________')
#         np_image_3 = np.array(small_pil_img)
# #         print(np_image_3.dtype, np_image_2.dtype)
        
#         print('Same?', (np_image_3 == np_image_2).all(), '\n')
#         a_m = run_pred(np_image_3)
#         sv.plot_image(a_m)
#         print(np_rot[0,0,0], '\n')
        
#         print('After retrieved\n')
        
#         print(np_image_2[0,0,0], '\n')
#         print("Mean of rotated image", np.mean(np_rot))
#         print("Mean of PILed image", np.mean(np_image_3))
#         print("Mean of retrieved image", np.mean(np_image_2), '\n')
        
#         diff = np.subtract(np_image_3, np_image_2)
        
#         diff = diff.astype('int8')
#         print(diff.dtype)
#         print('diff')
#         print(diff)
#         print(np.mean(diff))
#         wh = np.nonzero(diff)
#         print(wh)
    
# ________________________________________
    frame_out = av.VideoFrame.from_ndarray(np_image_2, format='bgr24')
    pkt = output_stream.encode(frame_out)
    outp.mux(pkt)
container.close()
outp.close()

print(f'Annotated video has been saved as {out_path}')
end_time = time.time()

Annotated video has been saved as GOPR1077_tr_annotated.mp4


In [85]:
print(round(end_time - start_time))
# without any modification == 41.5
# with modification == 10

10
